*To Do List:*

Add the screening histogram

Run it with new version of livingpark_utill

edit the downloader cell

add some explanation about H & Y 

Grammerly

Cohort ID?

Should I add edu, handness, etc.? 

This notebook tries to reproduce ICA components extraction part of the following paper. 
<div class="alert alert-block alert-success"> Zeighami, Yashar, et al. <a href=https://elifesciences.org/articles/08440>Network structure of brain atrophy in de
novo Parkinson’s disease.</a> eLife 4:e08440.
</div>

This study aimed to find the Parkinson's disease progression network. In this process, the authors first tried finding a pattern of volume atrophy in the brain. They used FFSL MELODIC to extract independent component analysis (ICA) components of the deformation-based morphometry (DBM) data. Then, they calculated DBM changes voxel vise in PD patients comparison to healthy subjects. They found significant correlation between medical measurements of disease severity and DBM changes. After that, comparing the pattern to a functioning network in a healthy brain, to see which brain network and function are affected, and based on that suggested a propagation model.

This study was conducted on PPMI dataset, and it used longitudinal MRI data from 232 de novo PD (i.e., patients not yet taking any medication) and 117 age-matched control subjects.
Clinical characteristics of subjects is shown in the below table (the table is extracted from the paper):

<img src="images/clinical characteristics.png" width=800/>

## Clinical Correlations
As mentioned earlier, two clinical measures were used to confirm the PD-ICA network. A berif descrption of these measures are provided in bellow:

* <span style="font-size:10.0pt;"> <b> Striatum binding ratio (SBR)</b>: Measures dopamine nerve terminal density.</span>
* <span style="font-size:10.0pt;"><b> Score on the Movement Disorder Society revised Unified Parkinson’s Disease Rating Scale (UPDRS) part III</b>: An objective measure of motor disability.</span>

There were significant correlation between both clinical measures, and each of measurments with DBM for PD group. 
The figures bellow -which extraxted from the paper- show correlations between the measurments and DBM of PD-ICA network.

<img src="images/clinical measures.png" width=600/>

In [14]:
#- <b>SBR & Age</b> is significant in the control group.
# - <b>SBR & DBM</b> is significant in the PD group; the greater the loss of dopamine nerve terminal, the greater volume loss in PD patients.
# - <b>UPDRS & DBM</b> is significant in the PD group.
# - <b>SBR & UPDRS</b> is significant in the PD group.

### Initial Setup
Initially we download and install dependecies.

In [15]:
import livingpark_utils
utils = livingpark_utils.LivingParkUtils()
utils.notebook_init()

removing link inputs
removing link outputs
This notebook was run on 2023-01-10 22:56:26 UTC +0000


Here some useful libraries, functions, and constant variables are loaded. The module `zeighamietal` from `livingpark_utils`, has helper functions and constants that are used in replication of both  [Zeighami *et al* (2015)](https://doi.org/10.7554/eLife.08440) and  [Zeighami *et al* (2017)](https://doi.org/10.7554/eLife.08440).

The code below creats a cohort based on T1 availability.


In [16]:
import re
from functools import reduce
import numpy as np
import pandas as pd

In [17]:
from livingpark_utils.zeighamietal.constants import (
    FILENAME_PARTICIPANT_STATUS,
    FILENAME_DEMOGRAPHICS,
    FILENAME_AGE,
    FILENAME_MOCA,
    FILENAME_UPDRS3,
    FILENAME_T1_INFO,
)

from livingpark_utils.zeighamietal.constants import (
    COL_PAT_ID,
    COL_STATUS,
    COL_VISIT_TYPE,
    COL_DATE_INFO,
)

from livingpark_utils.zeighamietal.constants import (
    STATUS_PD,
    STATUS_HC,
    MAIN_COHORT,
    VISIT_BASELINE,
    VISIT_SCREENING,
    SEX_MALE,
)

from livingpark_utils.zeighamietal.constants import (
    COL_PD_STATE,
    COL_AGE,
    COL_SEX,
    COL_EDUCATION,
    COL_MOCA,
    COLS_PIGD_COMPONENTS_UPDRS3,
    COL_FOLLOWUP,
)

from livingpark_utils.zeighamietal import (
    load_ppmi_csv,
    get_t1_cohort,
    mean_impute,
)

COL_UPDRS3 = ["NHY", "NP3TOT"] #columns that are needed from UPDRS3

In [ ]:

FILENAME_DEMOGRAPHICS = "Demographics.csv"
FILENAME_AGE = "Age_at_visit.csv"
FILENAME_PARTICIPANT_STATUS = "Participant_Status.csv"
FILENAME_MOCA = "Montreal_Cognitive_Assessment__MoCA_.csv"
FILENAME_UPDRS3 = "MDS_UPDRS_Part_III.csv"

### Downloading study data

We use `livingpark_utils` to download required files to build the cohort. If files be already available locally in the catch, the function skips downloading those particular files, nonetheless, the function asks for a PPMI username and password to aquire the files.

The used files for this cohort replication (based on the first tabel in the notebook)  are listed below:
* Participant status 

* Demographics

* Age at visit

* Clinical/cognitive assessment results:

    * Montreal Cognitive Assessment (MoCA)
    * Unified Parkinson's Disease Rating Scale (UPDRS) Part III

In [18]:
# Downloading needed files from PPMI for replicating the cohort

# utils.download_ppmi_metadata(["Demographics.csv"], headless=False)
                            # "Participant_Status.csv",
                            # "Age_at_visit.csv",
                            # "MDS_UPDRS_Part_III.csv",
                            # "Montreal_Cognitive_Assessment__MoCA_.csv"],headless=False)

# downloader = livingpark_utils.download.ppmi.Downloader(utils.study_files_dir)
# utils.get_study_files(required_files, default=downloader)

### Selecting subjects with available T1 MRI scan

For this study the authurs procliamed that they used initial visit 3T high-resolution T1-wheighted MRI scans of 237 PD patients and 118 age-matched control subjects from September 2013 to January 2014 (They excluded 5 PD patients and a control subject from analysis due to failiure in MRI processing). However, if we restrict time spane to aformentioned period, the number of available scans drops dramatically (38 PD patients and 1 helthy control). Instead, we used all subjects with available T1 scan with the magnetic field strength of 3T before February 1, 2014, and we obtained 236 PD patients and 113 healthy controls, which is closer to expected cohort.  

In [19]:
df_status = load_ppmi_csv(utils, FILENAME_PARTICIPANT_STATUS)

cohort_t1_map = {}
cohort_name = MAIN_COHORT

print(f"=============== {cohort_name.capitalize()} cohort ===============")

df_t1_subset = get_t1_cohort(
    utils,
    cohort_name=cohort_name,
    filename=FILENAME_T1_INFO,
    sagittal_only=True,
)
cohort_t1_map[cohort_name] = df_t1_subset

# cohort composition: number of PD patients/healthy controls
print(
    df_status.loc[
        df_status[COL_PAT_ID].isin(df_t1_subset[COL_PAT_ID]), COL_STATUS
    ].value_counts()
)

=============== Main cohort ===============
Removing extra scans for 1 subjects
Parkinson's Disease    236
Healthy Control        113
Name: COHORT_DEFINITION, dtype: int64


### Computing requierd clinical measures and imputing missing values

We imputed missing values for all data columns with mean of each column except for MoCA score. Baseline MoCA score has a lot of missing values, thus imputing values with mean of the column can not be informative. So, we filled the missing values with screening MoCA values.

We plot a histogram of days between baseline and screening visit MoCA, to make sure this attiude was reasonable.

In [42]:
cols_for_merge = [COL_PAT_ID, COL_DATE_INFO, COL_VISIT_TYPE]
df_updrs3 = load_ppmi_csv(
    utils, FILENAME_UPDRS3, cols_to_impute=COLS_PIGD_COMPONENTS_UPDRS3 + COL_UPDRS3
)
df_moca = load_ppmi_csv(utils, FILENAME_MOCA)  # do not impute

#df_updrs3 = df_updrs3.loc[df_updrs3[COL_PD_STATE] != "OFF"]

df_updrs3 = df_updrs3.loc[:, cols_for_merge + COL_UPDRS3]
df_moca = df_moca.loc[:, cols_for_merge + [COL_MOCA]]

df_assessments_all = reduce(
    lambda df1, df2: df1.merge(df2, on=cols_for_merge, how="outer"),
    [ df_updrs3, df_moca],
).drop_duplicates()

# some missing values remain even if we use the screening visit score
# we will impute these using the original mean
mean_moca = df_moca[COL_MOCA].mean()

cols_to_impute = [col for col in ["NHY", "NP3TOT", "MCATOT"] if col != COL_MOCA]
df_assessments_all = mean_impute(df_assessments_all, cols_to_impute)

# keep only subjects who have a T1
cohort_assessments_map_orig = {}
for cohort_name, df_t1_subset in cohort_t1_map.items():
    cohort_assessments_map_orig[cohort_name] = df_assessments_all.loc[
        df_assessments_all[COL_PAT_ID].isin(df_t1_subset[COL_PAT_ID])
    ]

In [50]:
col_date_diff = "date_diff"

cohort_assessments_map = {}
for cohort_name in cohort_assessments_map_orig:

    print(f"========== {cohort_name.upper()} COHORT ==========")

    date_diffs = []

    df_assessments_cohort: pd.DataFrame = cohort_assessments_map_orig[cohort_name]
    df_assessments_baseline = df_assessments_cohort.loc[
        df_assessments_cohort[COL_VISIT_TYPE] == VISIT_BASELINE
    ]
    df_assessments_screening = df_assessments_cohort.loc[
        df_assessments_cohort[COL_VISIT_TYPE] == VISIT_SCREENING
    ]

    # try to fill in missing baseline data
    for idx_row_baseline, row_baseline in df_assessments_baseline.iterrows():

        subject = row_baseline[COL_PAT_ID]
        date_baseline = row_baseline[COL_DATE_INFO]

        # for each score columns
        for col in [COL_MOCA]:
         
            # fill missing values with screening data
            if pd.isna(row_baseline[col]):

                df_screening_subject = df_assessments_screening.loc[
                    df_assessments_screening[COL_PAT_ID] == subject
                ]

                # in case we have more than one screening, wesort them by how close they are to the baseline visit
                n_screening = len(df_screening_subject)
                if n_screening > 1:
                    df_screening_subject[col_date_diff] = (
                        date_baseline - df_screening_subject[COL_DATE_INFO]
                    )
                    df_screening_subject = df_screening_subject.sort_values(
                        col_date_diff, ascending=True
                    )

                # find corresponding assessment score in screening visits
                for idx_row_screening, row_screening in df_screening_subject.iterrows():
                    new_value = row_screening[col]
                    date_diff = date_baseline - row_screening[COL_DATE_INFO]
                    if not pd.isna(new_value):
                        break

                # replace
                if not pd.isna(new_value):
                    df_assessments_baseline.loc[idx_row_baseline, col] = new_value
                    date_diffs.append(date_diff.days)  # for plotting
                    
    subjects_common = set(df_assessments_cohort[COL_PAT_ID])
    # print cohort composition
    print(
        df_status.loc[
            df_status[COL_PAT_ID].isin(subjects_common), COL_STATUS
        ].value_counts()
    )
    
    df_assessments_baseline[COL_FOLLOWUP] = False

    # impute remaining missing MoCA values
    df_assessments_baseline.loc[
        df_assessments_baseline[COL_MOCA].isna(), COL_MOCA
    ] = mean_moca

    cohort_assessments_map[cohort_name] = df_assessments_baseline



========== MAIN COHORT ==========
Parkinson's Disease    236
Healthy Control        113
Name: COHORT_DEFINITION, dtype: int64


In [52]:
def to_1_decimal_str(f):
    return str(round(f, 1))


df_age = load_ppmi_csv(utils, FILENAME_AGE)
df_demographics = load_ppmi_csv(utils, FILENAME_DEMOGRAPHICS)

col_male = "is_male"
col_cohort = "cohort"

dfs_summary = []
df_assessments: pd.DataFrame
for cohort_name, df_assessments in cohort_assessments_map.items():

    subjects = df_assessments[COL_PAT_ID].drop_duplicates()

    # general demographics (baseline session only)
    df_assessments =df_assessments.merge(df_status,on=[COL_PAT_ID],how="outer")
    df_summary = df_assessments.merge(df_age, on=[COL_PAT_ID, COL_VISIT_TYPE],how="outer")
    df_demographics[col_male] = (df_demographics[COL_SEX] == SEX_MALE).apply(
        lambda v: 100 if v else 0
    )

   
    df_summary = df_summary.merge(df_demographics, on=COL_PAT_ID,how="outer")
    df_summary = df_summary[[COL_PAT_ID, COL_AGE, col_male, COL_STATUS, COL_UPDRS3[1],COL_UPDRS3[0],COL_MOCA]]

    # append
    df_summary[col_cohort] = cohort_name
    dfs_summary.append(df_summary)

df_summary = pd.concat(dfs_summary)
df_summary = df_summary.iloc[np.where(df_summary[COL_STATUS].isin([STATUS_PD,STATUS_HC]))]
df_summary = df_summary.drop(columns=COL_PAT_ID)
df_summary_means = (
    df_summary.groupby([COL_STATUS]).mean().applymap(to_1_decimal_str))

df_summary_stds = (
    df_summary.groupby([COL_STATUS]).std().applymap(to_1_decimal_str)
)
df_summary_stds = " (" + df_summary_stds + ")"
df_summary_stds.loc[:, col_male] = ""
df_summary_combined = (df_summary_means + df_summary_stds).T
df_summary_combined = df_summary_combined.applymap(lambda x: "-" if "nan" in x else x)
df_summary_combined = df_summary_combined.rename(
    index={
        COL_AGE: "Age",
        col_male: "Male (%)",
        COL_UPDRS3[1]: "UPDRS Part III",
        COL_UPDRS3[0]: "H & Y",
        COL_MOCA: "MoCA",
    }
)
df_summary_combined


COHORT_DEFINITION,Healthy Control,Parkinson's Disease
Age,60.1 (11.3),61.2 (9.3)
Male (%),62.8,60.9
UPDRS Part III,1.2 (2.7),21.5 (8.9)
H & Y,0.0 (0.1),1.6 (0.5)
MoCA,28.3 (1.2),27.4 (2.2)


The upper tabel is the replicated cohort. The values are generally close to the main cohort, although they are not exactly equal. 